In [2]:
import logging.config
import configparser
from platform import python_version
import pandas as pd
import matplotlib.pyplot as plt

import tools.price_histories_helper as phh
import tools.alpha_factors_helper as afh
import tools.backtesting_functions as btf
import tools.configuration_helper as config_helper
import tools.utils as utils
import tools.ameritrade_functions as amc
import warnings

warnings.filterwarnings('ignore')

logging.config.fileConfig('./config/logging.ini')

logger = logging.getLogger('PortfolioHoldingsAdjustmentTesting')

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

logger.info(f'Python version: {python_version()}')
logger.info(f'Pandas version: {pd.__version__}')

config = configparser.ConfigParser()
config.read('./config/config.ini')
portfolio_config = config["DEFAULT"]

account = 'IRA'
account_config = config[account]
implemented_strategy = config_helper.get_implemented_strategy(account_config)
logger = logging.getLogger(f'GenerateAndSelectAlphaFactors.{implemented_strategy}.adjustment')
logger.info(f'Adjusting portfolio: {account}')
strategy_config = configparser.ConfigParser()
strategy_config.read('./data/' + 'weekly_low_risk_max15_final' + '/config.ini')
strategy_config = strategy_config['DEFAULT']
final_strategy_path = config_helper.get_final_strategy_path(strategy_config)
alpha_vectors = btf.load_alpha_vectors(config_helper.get_alpha_vectors_final_path(strategy_config))
daily_betas = btf.load_beta_factors(config_helper.get_daily_betas_final_path(strategy_config))
min_viable_return = strategy_config.getfloat('min_viable_port_return')
forward_prediction_days = strategy_config.getint('ForwardPredictionDays')
optimal_holdings_df = btf.predict_optimal_holdings(alpha_vectors,
                                                   daily_betas,
                                                   list(daily_betas.keys())[-1:],
                                                   risk_cap=strategy_config.getfloat('risk_cap'),
                                                   weights_max=strategy_config.getfloat('weights_max'),
                                                   weights_min=strategy_config.getfloat('weights_min'))

2023-02-05 10:38:34,247|BacktestingFunctions.predict_optimal_holdings|INFO|PARAMETERS|risk_cap|0.1|weights_max|0.15|weights_min|0.0
2023-02-05 10:38:34,248|BacktestingFunctions.predict_optimal_holdings|INFO|DATE_RANGE|2023-02-03 00:00:00|2023-02-03 00:00:00


Dates: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:26<00:00, 26.53s/ Portfolio Optimization]


In [24]:
td_ameritrade = amc.AmeritradeRest()
optimal_holdings = optimal_holdings_df.iloc[-1].round(2)
optimal_holdings.name = 'optimalWeights'
optimal_holdings = optimal_holdings[optimal_holdings > 0.05]
for index, value in optimal_holdings.items():
    logger.info(f'STOCK|{index:20}|HOLDING|{value:2f}')
for index, row in td_ameritrade.get_quotes(list(optimal_holdings.index.to_list())).iterrows():
    logger.info(f'QUOTES|{row.symbol}|{row.assetMainType}|CUSIP|{row.cusip}|' +
                f'BID/ASK|{row.bidPrice}/{row.askPrice}|CHANGE|{row.regularMarketNetChange}' +
                f'|{row.description}')
td_ameritrade.refresh_data()
masked_account_number = config_helper.get_masked_account_number(account_config)
current_cash_balance = td_ameritrade.parse_accounts().loc[masked_account_number].currentBalances_cashBalance
cash_equivalents_balance = td_ameritrade.get_account_portfolio_data(masked_account_number,
                                                                    'CASH_EQUIVALENT').marketValue.sum()
total_amount_available = current_cash_balance + cash_equivalents_balance
logger.info(f'ACCOUNT|{masked_account_number}|CASH_VALUE|{current_cash_balance}|' +
            f'CASH_EQUIV|{cash_equivalents_balance}|' +
            f'AVAILABLE|{total_amount_available}')
# TODO: Make stock trades
trade_configurations_df = optimal_holdings.to_frame()
trade_configurations_df['Amount'] = (trade_configurations_df.optimalWeights * total_amount_available).round(0)
quotes_df = td_ameritrade.get_quotes(list(trade_configurations_df.index.to_list()))[['assetType', 'bidPrice', 'askPrice', 'regularMarketLastPrice']]
trade_configurations_df = pd.concat([trade_configurations_df, quotes_df], axis=1)
trade_configurations_df['Quantity'] = (trade_configurations_df.Amount / trade_configurations_df.regularMarketLastPrice).round(0)


for index, row in trade_configurations_df.iterrows():
    logger.info(f'STOCK|{index:20}|HOLDING|{row.optimalWeights:2f}|{row.Amount}')
# TODO: If you want to take the amount to invest down to the penny, you could get trading costs and subtract from
#       amount

In [27]:
trade_configurations_df

,optimalWeights,Amount,assetType,bidPrice,askPrice,regularMarketLastPrice,Quantity
CAH,0.15,4163.0,EQUITY,78.21,134.36,78.67,53.0
CEG,0.15,4163.0,EQUITY,81.00,84.90,83.20,50.0
ENPH,0.15,4163.0,EQUITY,222.90,223.43,222.93,19.0
FSLR,0.15,4163.0,EQUITY,168.25,168.82,168.19,25.0
LW,0.10,2776.0,EQUITY,96.48,99.46,98.12,28.0
NUE,0.15,4163.0,EQUITY,176.00,178.64,176.63,24.0
STLD,0.15,4163.0,EQUITY,127.50,129.50,128.18,32.0


In [23]:
pd.concat([trade_configurations_df, quotes_df], axis=1)

,optimalWeights,Amount,assetType,bidPrice,askPrice,regularMarketLastPrice
CAH,0.15,4163.0,EQUITY,78.21,134.36,78.67
CEG,0.15,4163.0,EQUITY,81.00,84.90,83.20
ENPH,0.15,4163.0,EQUITY,222.90,223.43,222.93
FSLR,0.15,4163.0,EQUITY,168.25,168.82,168.19
LW,0.10,2776.0,EQUITY,96.48,99.46,98.12
NUE,0.15,4163.0,EQUITY,176.00,178.64,176.63
STLD,0.15,4163.0,EQUITY,127.50,129.50,128.18
